In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt


df_original = pd.read_csv('events.csv', low_memory=False, date_parser=pd.datetools.to_datetime)

In [ ]:
ad_campaign_df_2 = df_original.loc[df_original['event']=='ad campaign hit', :].dropna(axis=1, how='all')
# display(ad_campaign_df_2.head())
# display(ad_campaign_df_2['campaign_source'].value_counts())
ad_campaign_df_2 = pd.DataFrame(ad_campaign_df_2['campaign_source'].value_counts())
ad_campaign_df_2.reset_index(inplace=True)
ad_campaign_df_2.columns = ['campaign_source', 'hits']
ad_campaign_df_2.set_index('campaign_source', inplace=True)

In [ ]:
ad_campaign_df = df_original.loc[df_original['event']=='ad campaign hit', :].dropna(axis=1, how='all')

def datetime_to_month(str_date):
    return int(str(str_date).split('-')[1])

def datetime_to_day(str_date):
    return int(str(str_date).split('-')[2])

ad_campaign_df['month_hit'] = ad_campaign_df['timestamp'].agg(datetime_to_month)
grouped = ad_campaign_df.groupby('campaign_source').agg({'month_hit':['min', 'mean', 'std', 'max', 'median']})
grouped.columns = ['min_month_hit', 'mean_month_hit','std_month_hit', 'max_month_hit', 'median_hit']
grouped.reset_index(inplace=True)
grouped.set_index('campaign_source', inplace=True)

In [ ]:
campaigns_and_hits = grouped.join(ad_campaign_df_2)
campaigns_and_hits.sort_values(by=['median_hit', 'hits'], inplace=True)
campaigns_and_hits

Vemos que la mediana de las campañas mas importantes (que mas gente llevaron al citio) ocurre en el mes 5. Lo que nos dice que entre los meses 1,2,3,4 y una parte del 5 tenemos la misma cantidad de hits (por campaña) que en los meses 6 y una parte del 5.

Esto coincide con lo observado en los gráficos anteriores, que el citio tuvo una suba de tráfico durante el mes de junio.

In [ ]:
ad_campaign_df['month_hit'] = ad_campaign_df['timestamp'].agg(datetime_to_month)
grouped_2 = ad_campaign_df.groupby(['campaign_source', 'month_hit']).agg({'person':'count'})
grouped_2
grouped_2_reshaped = pd.pivot_table(grouped_2, values='person', columns='month_hit', index='campaign_source').replace(np.nan, 0)

In [ ]:
grouped_2_reshaped

Viendo esta tabla, podemos corroborar que la mayoría de las campañas claramente tuvieron una alza en el trafico hacia el citio en el mes 5. Que coincide con lo analizado previamente.

In [ ]:
ad_campaign_df = ad_campaign_df[['timestamp', 'campaign_source']]

In [ ]:
ad_campaign_df['day_of_year'] = pd.to_datetime(ad_campaign_df['timestamp']).dt.dayofyear

In [ ]:
ad_campaign_df.head()

In [ ]:
ad_campaign_df.head()

In [ ]:
google_hits = ad_campaign_df.loc[ad_campaign_df['campaign_source']=='google', :]['day_of_year']
criteo_hits = ad_campaign_df.loc[ad_campaign_df['campaign_source']=='criteo', :]['day_of_year']
rtbhouse_hits = ad_campaign_df.loc[ad_campaign_df['campaign_source']=='rtbhouse', :]['day_of_year']
zanox_hits = ad_campaign_df.loc[ad_campaign_df['campaign_source']=='zanox', :]['day_of_year']

In [ ]:
plt.subplots(figsize=(10, 10))
google_hits.plot.hist(bins=50, alpha=0.3)
criteo_hits.plot.hist(bins=50, alpha=0.3)
rtbhouse_hits.plot.hist(bins=50, alpha=0.3)
zanox_hits.plot.hist(bins=50, alpha=0.3)
plt.xlabel('Días del año', fontsize=15)
plt.ylabel('Hits a la página', fontsize=15) 
plt.title('Distribución de hits a la página desde campañas publicitarias (Período de enero-junio)', fontsize=20)
plt.legend(['Google', 'Criteo', 'RTBHouse', 'Zanox'])
plt.grid()